In [12]:
import numpy as np
import pandas as pd
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import  BottomUp, TopDown, MinTrace
from statsforecast.core import StatsForecast
from statsforecast.models import AutoETS
from sklearn.metrics import mean_absolute_error


sales_train_eval = pd.read_csv('sales_train_evaluation.csv')
sell_price = pd.read_csv('sell_prices.csv')
calendar = pd.read_csv('calendar.csv')

foods = pd.read_csv('List_of_foods.csv')
n=5



MASE_errors_average = [0] * 9

for food_num in range(n):
    product_id = foods.loc[food_num].at["Foods"]


    product_data = sales_train_eval[sales_train_eval['item_id'].str.contains(product_id)]
    product_sell_price = sell_price[sell_price['item_id'].str.contains(product_id)]


    df = pd.melt(
        product_data,
        id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
        var_name='d',
        value_name='value').dropna()
    df = pd.merge(df, calendar, on='d', how='left')


    df = df[(df['date'] > '2016-01-01')]



    df_stores = df.groupby(['date', 'store_id'])[['value']].sum()
    df_stores.reset_index(inplace=True)
    df_stores = df_stores.T.reset_index(drop=True).T
    df_stores.columns = ['d', 'unique_id', 'sales']


    df_state = df.groupby(['date', 'state_id'])[['value']].sum()
    df_state.reset_index(inplace=True)
    df_state = df_state.T.reset_index(drop=True).T
    df_state.columns = ['d', 'unique_id', 'sales']


    df_total = df.groupby(['date'])[['value']].sum()
    df_total.reset_index(inplace=True)
    df_total['unique_id'] = 'Total'
    df_total.columns = ['d','sales', 'unique_id']

    df_all = pd.concat([df_stores, df_state, df_total], axis = 0)

    xset = set(df_all.unique_id)
    df_all.columns = ['ds', 'unique_id', 'y']
    df_all['ds'] = pd.to_datetime(df_all['ds'])






    #making the summing matrix

    S = np.zeros((len(xset), len([f for f in xset if '_' in f])))


    # rows / columns
    list1 = ['Total', 'CA','CA_1','CA_2','CA_3','CA_4','TX','TX_1','TX_2','TX_3','WI','WI_1','WI_2','WI_3']
    list2 = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
    S = pd.DataFrame(S); S.index = list1; S.columns = list2


    # encode the hierarchical structure
    S.loc['Total'] = 1
    S.loc['CA'][['CA_1','CA_2','CA_3', 'CA_4']] = 1
    S.loc['TX'][['TX_1','TX_2','TX_3']] = 1
    S.loc['WI'][['WI_1','WI_2','WI_3']] = 1
    for x in S.columns:
        S.loc[x][x]= 1
    S = S.astype(int)
    S




    tags = {}
    tags['Country'] = np.array(['Total'], dtype=object)
    tags['Country/State'] = np.array(['CA', 'TX', 'WI'], dtype=object)
    tags['Country/State/Store'] = np.array(['CA_1', 'CA_2', 'CA_3', 'CA_4',  
                                            'TX_1', 'TX_2', 'TX_3',
                                            'WI_1', 'WI_2', 'WI_3'], dtype=object)
    tags






    horizon = 28



    x_test = df_all.groupby('unique_id').tail(horizon)
    x_train = df_all.drop(x_test.index)
    x_test = x_test.set_index('unique_id')
    x_train = x_train.set_index('unique_id')





    fcst = StatsForecast(df = x_train, models=[AutoETS(season_length= 7, model ='AAA', damped = True)], freq='D', n_jobs=-1)
    x_hat = fcst.forecast(h = horizon)



    def MASE(y_true, y_pred, y_train):
        e_t = y_true - y_pred
        scale = mean_absolute_error(y_train[1:], y_train[:-1])
        return np.mean(np.abs(e_t / scale))




    reconcilers = [
        MinTrace(method='ols'),
        TopDown(method='forecast_proportions'),
        BottomUp()
    ]

    hrec = HierarchicalReconciliation(reconcilers=reconcilers)

    x_hat_rec = hrec.reconcile(x_hat, S, tags)




    #methods = ['ETS/MinTrace_method-ols','ETS/TopDown_method-forecast_proportions','ETS/BottomUp']
    methods = ['AutoETS/MinTrace_method-ols','AutoETS/TopDown_method-forecast_proportions','AutoETS/BottomUp']
    
    nm=len(methods)

    errors = [0] * 9

    counter = 0
    for i in range(nm):

        rightdf = x_hat_rec[["ds",methods[i]]]
        xmat = pd.merge(left = x_test, right = rightdf, on = ['ds', 'unique_id'])
        xmat.columns = [['ds', 'y', 'pred']]

        
        for k in tags.keys():
            errors[counter] = MASE(xmat.loc[tags[k]]['y'].to_numpy(), xmat.loc[tags[k]]['pred'].to_numpy(), x_train.loc[tags[k]]['y'].to_numpy())
            counter += 1

    
    MASE_errors_average = np.array(MASE_errors_average) + np.array(errors)  


MASE_errors_average = MASE_errors_average/5


count=0
for i in range(nm):
    print('MASE for ' + methods[i])
    for k in tags.keys():
        print(k + ' MASE: ' + str(MASE_errors_average[count]))
        count += 1
    print("")


MASE for AutoETS/MinTrace_method-ols
Country MASE: 0.48465810432292
Country/State MASE: 0.6372108611490623
Country/State/Store MASE: 0.8103811659752402

MASE for AutoETS/TopDown_method-forecast_proportions
Country MASE: 0.47959476296731746
Country/State MASE: 0.650941928889983
Country/State/Store MASE: 0.80989158924257

MASE for AutoETS/BottomUp
Country MASE: 0.5329773653878881
Country/State MASE: 0.6460303340254733
Country/State/Store MASE: 0.8096337104851911

